In [134]:
import pandas as pd
import numpy as np

In [135]:
data=pd.read_csv("C:\\Users\\DELL\\Desktop\\semester_3\\modelling in operation management\\market analysis\\data\\csi.csv")
len(data)

1268

In [136]:
data.isnull().sum()

Date          0
Open         62
High         62
Low          62
Close        62
Adj Close    62
Volume       62
dtype: int64

In [137]:
data.fillna(data.mean(), inplace=True)

In [138]:
data.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [139]:
def MA(data, n):  
    MA = pd.Series(data['Close'].rolling(window = n).mean(), name = 'MA' + str(n))  
    data = data.join(MA) 
    return data


In [140]:
data=MA(data,5)
data=MA(data,10)
data=MA(data,20)

In [141]:
def EMA(data, n):  
    EMA = pd.Series(data['Close'].ewm(span = n, min_periods = n - 1).mean(), name = 'EMA_' + str(n))  
    return EMA

In [142]:
data['DIF']=EMA(data,12)-EMA(data,26)


In [143]:
def MACD(data,n):
    MACD=pd.Series(data['DIF'].ewm(span = n, min_periods = n-1).mean(), name = 'MACD')
    data=data.join(MACD)
    return data


In [144]:
data=MACD(data,9)

In [145]:
def stochastics( data, low, high, close, k, d ):
    # Set minimum low and maximum high of the k stoch
    low_min  = data[low].rolling( window = k ).min()
    high_max = data[high].rolling( window = k ).max()

   
    data['KDJ.K'] = 100 * (data[close] - low_min)/(high_max - low_min)
    data['KDJ.D'] = data['KDJ.K'].rolling(window = d).mean()

    return data

In [146]:
data = stochastics( data, 'Low', 'High', 'Close', 14, 3)

In [147]:
def PSY(data,Length):
    psy=[]
    for i in range(5,len(data['Close'])):
        c=0
        for j in range(i,i-5,-1):
            if data['Close'][j] > data['Close'][j-1]:
                c+=1
        psy.append(c/Length)        
    return psy

In [148]:
PSYMA6=PSY(data,6)
psy=[np.NaN]*5
PSYMA6=psy+PSYMA6
data['PSYMA6']=PSYMA6

In [149]:
def RSI(data, n):
    delta = data.Close.diff()
    window = n
    up_days = delta.copy()
    up_days[delta<=0]=0.0
    down_days = abs(delta.copy())
    down_days[delta>0]=0.0
    RS_up = up_days.rolling(window).mean()
    RS_down = down_days.rolling(window).mean()
    rsi= 100-100/(1+RS_up/RS_down)
    
    return rsi

In [150]:
rsi6=RSI(data,6)
rsi12=RSI(data,12)
data['RSI6']=rsi6
data['RSI12']=rsi12

In [151]:
def Bias(data,n):
    sma=data['Close'].rolling( window = n).mean()
    bias=((data['Close']-sma)/sma)*100  
    return bias

In [152]:
BIAS6=Bias(data,6)
BIAS24=Bias(data,24)
data['BIAS6']=BIAS6
data['BIAS24']=BIAS24

In [153]:
file=data.to_csv("finalDataset.csv",index=False,header=True)